In [1]:
!pip install opencv-python
!pip install opencv-python-headless

from PIL import Image
from IPython.display import display
import os
import xml.etree.ElementTree as gfg
import pandas as pd
import cv2
import numpy as np
import time


In [2]:
#first get the detections from openimages
detections = pd.read_csv("C:/Users/paperspace/fiftyone/open-images-v6/train/labels/detections.csv")

count_row = detections.shape[0]  # Gives number of rows
count_col = detections.shape[1]  # Gives number of columns

print(count_row)
print(count_col)


#get the classes from openimages
classes =  pd.read_csv("C:/Users/paperspace/fiftyone/open-images-v6/train/metadata/classes.csv")

classes.columns =['code', 'name']

count_row_classes = classes.shape[0]  # Gives number of rows
count_col_classes = classes.shape[1]  # Gives number of columns

print(count_row_classes)
print(count_col_classes)


14610229
21
600
2


In [3]:
#our classes
my_classes = ["Guitar"]

my_classes_counters = [["Guitar",0]]

directory = "C:/Users/paperspace/fiftyone/open-images-v6/train/data"
counter = 0

In [4]:
def GenerateXML(filename,img):

    global counter   
    root = gfg.Element("annotation")
      
    _folder = gfg.SubElement(root, "folder")
    _folder.text = "data"
   
    _filename = gfg.SubElement(root,"filename")
    _filename.text = filename
   
    _path = gfg.SubElement(root,"path")
    _path.text = directory+"/"+filename
  
    _source = gfg.SubElement(root,"source")
    _database = gfg.SubElement(_source,"source")
    _database.text = "Unknown"
    
    #height, width, channels = img.shape

    width, height = img.size
    channels = 3

    _size = gfg.SubElement(root,"size")
    _width = gfg.SubElement(_size,"width")
    _width.text = str(width)
    _height = gfg.SubElement(_size,"height")
    _height.text = str(height)
    _depth = gfg.SubElement(_size,"depth")
    _depth.text = str(channels)
    
    _segmented = gfg.SubElement(root,"segmented")
    _segmented.text = "0"
    
    #now start searching for the image in the detections (might be more than one)
    subsetdetections = detections.loc[detections['ImageID'] == os.path.splitext(filename)[0]]

    classes_to_chceck = []
    for cls in my_classes:
        my_struct = [cls,False]
        classes_to_chceck.append(my_struct)
   
    for index, row in subsetdetections.iterrows():
       
        image = row['ImageID']
        labelname = classes.loc[classes['code'] == row['LabelName']].iat[0,1]
        #print(image,labelname)
        #now for this image we have to search for only one occurance of any of our classes

        for x in range(len(classes_to_chceck)):
            if classes_to_chceck[x][0] == labelname:
                if classes_to_chceck[x][1] == False:
                    classes_to_chceck[x][1] = True
                    #print(classes_to_chceck[x]) 
                    
                    #update counters
                    my_classes_counters[x][1] = my_classes_counters[x][1]+1

                    #if classes_to_chceck[x][0] == "Apple":
                    #    apple_counter = apple_counter+1
                    #if classes_to_chceck[x][0] == "Mug":
                    #   mug_counter = mug_counter+1
                    #if classes_to_chceck[x][0] == "Tin can":
                    #   tin_can_counter = tin_can_counter+1

                    #get the bounding box detections
                    perc_xmin = float(width)*float(row['XMin'])
                    perc_xmax = float(width)*float(row['XMax'])
                    perc_ymin = float(height)*float(row['YMin'])
                    perc_ymax = float(height)*float(row['YMax'])
                
                    #opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                    #cv2.rectangle(opencvImage, (int(perc_xmin),int(perc_ymin)),(int(perc_xmax),int(perc_ymax)), (255,0,0), 2)
                    #img_r = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
                    #cv2.imshow("result", img_r)
                    #cv2.waitKey(0)
                    #cv2.destroyAllWindows()

                    _object = gfg.SubElement(root,"object")
                    _name = gfg.SubElement(_object,"name")
                    _name.text = labelname
                    _pose = gfg.SubElement(_object,"pose")
                    _pose.text = "Unspecified"
                    _truncated = gfg.SubElement(_object,"truncated")
                    _truncated.text = "0"
                    _difficult = gfg.SubElement(_object,"difficult")
                    _difficult.text = "0"
                    _bndbox = gfg.SubElement(_object,"bndbox")
                    _xmin =  gfg.SubElement(_bndbox,"xmin")
                    _ymin =  gfg.SubElement(_bndbox,"ymin")
                    _xmax =  gfg.SubElement(_bndbox,"xmax")
                    _ymax =  gfg.SubElement(_bndbox,"ymax")
                    _xmin.text = str(int(perc_xmin))
                    _ymin.text = str(int(perc_ymin))
                    _xmax.text = str(int(perc_xmax))
                    _ymax.text = str(int(perc_ymax))
                
    tree = gfg.ElementTree(root)
    print("write xml: ",os.path.splitext(filename)[0]+'.xml')
    with open (directory+"/"+os.path.splitext(filename)[0]+'.xml' , "wb") as files :
        tree.write(files)

In [5]:
def process_images(folder):    
    global counter 
    for filename in os.listdir(folder):
        if os.path.splitext(filename)[1] != ".xml":
            counter = counter+1
            print(counter)
            if counter > 10000:
                #img = cv2.imread(os.path.join(folder,filename))
                img = Image.open(os.path.join(folder,filename))
                if img is not None:
                    GenerateXML(filename,img)
                    print("**********************************")
                    for x in range(len(my_classes_counters)):
                        print(my_classes_counters[x][0],my_classes_counters[x][1])
              

In [6]:
counter = 0
process_images(directory)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


8042
8043
8044
8045
8046
8047
8048
8049
8050
8051
8052
8053
8054
8055
8056
8057
8058
8059
8060
8061
8062
8063
8064
8065
8066
8067
8068
8069
8070
8071
8072
8073
8074
8075
8076
8077
8078
8079
8080
8081
8082
8083
8084
8085
8086
8087
8088
8089
8090
8091
8092
8093
8094
8095
8096
8097
8098
8099
8100
8101
8102
8103
8104
8105
8106
8107
8108
8109
8110
8111
8112
8113
8114
8115
8116
8117
8118
8119
8120
8121
8122
8123
8124
8125
8126
8127
8128
8129
8130
8131
8132
8133
8134
8135
8136
8137
8138
8139
8140
8141
8142
8143
8144
8145
8146
8147
8148
8149
8150
8151
8152
8153
8154
8155
8156
8157
8158
8159
8160
8161
8162
8163
8164
8165
8166
8167
8168
8169
8170
8171
8172
8173
8174
8175
8176
8177
8178
8179
8180
8181
8182
8183
8184
8185
8186
8187
8188
8189
8190
8191
8192
8193
8194
8195
8196
8197
8198
8199
8200
8201
8202
8203
8204
8205
8206
8207
8208
8209
8210
8211
8212
8213
8214
8215
8216
8217
8218
8219
8220
8221
8222
8223
8224
8225
8226
8227
8228
8229
8230
8231
8232
8233
8234
8235
8236
8237
8238
8239
8240
8241


write xml:  8c4b4da98c618aff.xml
**********************************
Guitar 1
10002
write xml:  8c4cc5a7ab2f5ed2.xml
**********************************
Guitar 2
10003
write xml:  8c5bf63c84629db7.xml
**********************************
Guitar 3
10004
write xml:  8c5d3250d16e44cf.xml
**********************************
Guitar 4
10005
write xml:  8c6009909d8946b4.xml
**********************************
Guitar 5
10006
write xml:  8c60c5217a185546.xml
**********************************
Guitar 6
10007
write xml:  8c64d8cdd88d62dc.xml
**********************************
Guitar 7
10008
write xml:  8c672010a59339ca.xml
**********************************
Guitar 8
10009
write xml:  8c698b98236d5565.xml
**********************************
Guitar 9
10010
write xml:  8c6aec818692769e.xml
**********************************
Guitar 10
10011
write xml:  8c6c3cf6e12c592e.xml
**********************************
Guitar 11
10012
write xml:  8c6e0d358bfc8d44.xml
**********************************
Guitar 12
10013
w

write xml:  8db8e9e7370db05d.xml
**********************************
Guitar 99
10100
write xml:  8db9f01bafaccc63.xml
**********************************
Guitar 100
10101
write xml:  8dbafe2760bb8bfb.xml
**********************************
Guitar 101
10102
write xml:  8dbc7dcf746075ea.xml
**********************************
Guitar 102
10103
write xml:  8dc4db768c6cb588.xml
**********************************
Guitar 103
10104
write xml:  8dcd8cbddc10438f.xml
**********************************
Guitar 104
10105
write xml:  8dd3476b1db82731.xml
**********************************
Guitar 105
10106
write xml:  8dd8285917de2291.xml
**********************************
Guitar 106
10107
write xml:  8ddc9a7a87085586.xml
**********************************
Guitar 107
10108
write xml:  8ddfbc5380a33742.xml
**********************************
Guitar 108
10109
write xml:  8de6bd2ad298321d.xml
**********************************
Guitar 109
10110
write xml:  8de719312b6ea3ee.xml
*********************************

write xml:  8f06f22169c4b6da.xml
**********************************
Guitar 196
10197
write xml:  8f0d09b2a6d696df.xml
**********************************
Guitar 197
10198
write xml:  8f16f17da5130234.xml
**********************************
Guitar 198
10199
write xml:  8f17793656890a3d.xml
**********************************
Guitar 199
10200
write xml:  8f19e112281818b5.xml
**********************************
Guitar 200
10201
write xml:  8f23f29a23b1a8e9.xml
**********************************
Guitar 201
10202
write xml:  8f267515fd6db924.xml
**********************************
Guitar 202
10203
write xml:  8f2aa170df272034.xml
**********************************
Guitar 203
10204
write xml:  8f352e09675fea05.xml
**********************************
Guitar 204
10205
write xml:  8f39430540f666a6.xml
**********************************
Guitar 205
10206
write xml:  8f3abee84d981f19.xml
**********************************
Guitar 206
10207
write xml:  8f3ddfa1175bc2f4.xml
********************************

...

write xml:  f924a044a599b85e.xml
**********************************
Guitar 7212
17213
write xml:  f925167a782027ec.xml
**********************************
Guitar 7213
17214
write xml:  f9348bfe5ab0fbcc.xml
**********************************
Guitar 7214
17215
write xml:  f936b0b4b52f1ab6.xml
**********************************
Guitar 7215
17216
write xml:  f9378778c6ee8152.xml
**********************************
Guitar 7216
17217
write xml:  f939b7f25480d7c4.xml
**********************************
Guitar 7217
17218
write xml:  f940e38d2ce04705.xml
**********************************
Guitar 7218
17219
write xml:  f9503c74002c1ea1.xml
**********************************
Guitar 7219
17220
write xml:  f952937e8982ad51.xml
**********************************
Guitar 7220
17221
write xml:  f955287b61036ef9.xml
**********************************
Guitar 7221
17222
write xml:  f95636de1e91f1e8.xml
**********************************
Guitar 7222
17223
write xml:  f95c679facb590c3.xml
*********************

write xml:  fa66ae3a0f105e40.xml
**********************************
Guitar 7308
17309
write xml:  fa682b263ecce088.xml
**********************************
Guitar 7309
17310
write xml:  fa703fa6c48ce7ee.xml
**********************************
Guitar 7310
17311
write xml:  fa7e69689e3a5e2f.xml
**********************************
Guitar 7311
17312
write xml:  fa8c09f065707cb2.xml
**********************************
Guitar 7312
17313
write xml:  fa90433feca344bc.xml
**********************************
Guitar 7313
17314
write xml:  fa91d000b45d75e0.xml
**********************************
Guitar 7314
17315
write xml:  fa9574026eaffe83.xml
**********************************
Guitar 7315
17316
write xml:  fa95f9034d9290f1.xml
**********************************
Guitar 7316
17317
write xml:  fa9865e3978ff020.xml
**********************************
Guitar 7317
17318
write xml:  fa99e6857a1b5e8a.xml
**********************************
Guitar 7318
17319
write xml:  fa9d3105e4b1cfe9.xml
*********************

write xml:  fbd9eeca2ac8b2d7.xml
**********************************
Guitar 7404
17405
write xml:  fbe588c16b8befe1.xml
**********************************
Guitar 7405
17406
write xml:  fbeb20b1e88dcf42.xml
**********************************
Guitar 7406
17407
write xml:  fbf99f085e488c60.xml
**********************************
Guitar 7407
17408
write xml:  fbfa6779db4a783e.xml
**********************************
Guitar 7408
17409
write xml:  fc08626ab224c59b.xml
**********************************
Guitar 7409
17410
write xml:  fc0a86f6d386a2a8.xml
**********************************
Guitar 7410
17411
write xml:  fc0d50fced940752.xml
**********************************
Guitar 7411
17412
write xml:  fc10765cf09fbe83.xml
**********************************
Guitar 7412
17413
write xml:  fc11672558aef67a.xml
**********************************
Guitar 7413
17414
write xml:  fc12a3c1795d288e.xml
**********************************
Guitar 7414
17415
write xml:  fc157b95655fc07c.xml
*********************

write xml:  fd975efba3547efa.xml
**********************************
Guitar 7500
17501
write xml:  fd997769807f3718.xml
**********************************
Guitar 7501
17502
write xml:  fd9bf57eed4de471.xml
**********************************
Guitar 7502
17503
write xml:  fd9ea032c440cbf9.xml
**********************************
Guitar 7503
17504
write xml:  fda01fa3812abd08.xml
**********************************
Guitar 7504
17505
write xml:  fda0ed93e4df6103.xml
**********************************
Guitar 7505
17506
write xml:  fda19afd6d4aca81.xml
**********************************
Guitar 7506
17507
write xml:  fda4cd038678faf3.xml
**********************************
Guitar 7507
17508
write xml:  fda85061f65c7931.xml
**********************************
Guitar 7508
17509
write xml:  fdaf7d1565e7bdf5.xml
**********************************
Guitar 7509
17510
write xml:  fdb3d93bd5a1748a.xml
**********************************
Guitar 7510
17511
write xml:  fdb90630a0b0a9f8.xml
*********************

write xml:  fed620c7f1b01040.xml
**********************************
Guitar 7596
17597
write xml:  fed705b6cbc1b852.xml
**********************************
Guitar 7597
17598
write xml:  fee090ea39231f64.xml
**********************************
Guitar 7598
17599
write xml:  fee487d594b03766.xml
**********************************
Guitar 7599
17600
write xml:  feeb557a30e01d06.xml
**********************************
Guitar 7600
17601
write xml:  feece97d022db73c.xml
**********************************
Guitar 7601
17602
write xml:  feed2b0b7fdbf785.xml
**********************************
Guitar 7602
17603
write xml:  feee2b691f398090.xml
**********************************
Guitar 7603
17604
write xml:  fef49512f71d2fb8.xml
**********************************
Guitar 7604
17605
write xml:  fef587d083923c9d.xml
**********************************
Guitar 7605
17606
write xml:  fefbf9077a70e0ce.xml
**********************************
Guitar 7606
17607
write xml:  fefe563294a01de4.xml
*********************